#pip

In [1]:
!unzip /content/drive/MyDrive/Colab_Notebooks/2023_AIfactory_위성segmentation_202307/정유탱크/oiltank_dataset.zip

Archive:  /content/drive/MyDrive/Colab_Notebooks/2023_AIfactory_위성segmentation_202307/정유탱크/oiltank_dataset.zip
   creating: oiltank_dataset/
   creating: oiltank_dataset/valid_images/
  inflating: oiltank_dataset/valid_images/OBJ05625_PS4_K3A_NIA0358.tif  
  inflating: oiltank_dataset/valid_images/OBJ05308_PS3_K3A_NIA0344.kml  
  inflating: oiltank_dataset/valid_images/OBJ03913_PS3_K3A_NIA0165.png  
  inflating: oiltank_dataset/valid_images/OBJ05658_PS3_K3A_NIA0359.kml  
  inflating: oiltank_dataset/valid_images/OBJ05155_PS4_K3A_NIA0336.tif  
  inflating: oiltank_dataset/valid_images/OBJ03972_PS4_K3A_NIA0166.tif  
  inflating: oiltank_dataset/valid_images/OBJ04926_PS3_K3A_NIA0172.png  
  inflating: oiltank_dataset/valid_images/OBJ05436_PS3_K3A_NIA0165.kml  
  inflating: oiltank_dataset/valid_images/OBJ05692_PS4_K3A_NIA0361.tif  
  inflating: oiltank_dataset/valid_images/OBJ05004_PS4_K3A_NIA0328.tif  
  inflating: oiltank_dataset/valid_images/OBJ03156_PS3_K3_NIA0128.png  
  inf

In [2]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 629.1/629.1 kB 11.4 MB/s eta 0:00:00


#import

In [1]:
import os
os.environ["WANDB_DISABLED"] = "true"

import pandas as pd
import numpy as np
import cv2
import shutil
import yaml
import warnings
warnings.filterwarnings("ignore")

from ultralytics import YOLO
from glob import glob
from tqdm import tqdm
from IPython.display import clear_output
from sklearn.model_selection import train_test_split

#data

In [7]:
train_df = pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/2023_AIfactory_위성segmentation_202307/정유탱크/train_data.csv')
train_df

,Unnamed: 0,index,train_data_loc,train_label_xy
0,0,0,OBJ02634_PS3_K3_NIA0126.png,"[(336.6415449432134, 339.1612715196364), (349...."
1,1,0,OBJ02634_PS3_K3_NIA0126.png,"[(322.5874084582725, 352.4346226443028), (337...."
2,2,0,OBJ02634_PS3_K3_NIA0126.png,"[(312.82759145484135, 367.2695444895181), (329..."
3,3,0,OBJ02634_PS3_K3_NIA0126.png,"[(287.8424599260575, 390.3027126176158), (301...."
4,4,0,OBJ02634_PS3_K3_NIA0126.png,"[(305.41013053223367, 394.59703209912556), (32..."
...,...,...,...,...
1005,1005,67,OBJ05733_PS3_K3A_NIA0363.png,"[(946.3780053972047, 128.88179847646327), (959..."
1006,1006,68,OBJ05760_PS3_K3A_NIA0365.png,"[(578.1284550609116, 225.30518552949212), (593..."
1007,1007,69,OBJ05781_PS3_K3A_NIA0368.png,"[(86.25686800677065, 264.84640029489117), (98...."
1008,1008,69,OBJ05781_PS3_K3A_NIA0368.png,"[(109.66569892916438, 271.8905213728775), (121..."


In [8]:
if not os.path.exists('data/yolo'):
  os.makedirs('data/yolo')

if os.path.exists("data/yolo"):
    shutil.rmtree("data/yolo")

if not os.path.exists("data/yolo/train"):
    os.makedirs("data/yolo/train")

if not os.path.exists("data/yolo/valid"):
    os.makedirs("data/yolo/valid")

if not os.path.exists("data/yolo/test"):
    os.makedirs("data/yolo/test")

if not os.path.exists("results"):
    os.makedirs("results")

In [9]:
import ast

def make_yolo_dataset(image_paths, txt_paths, _type="train"):
    for image_path, labels_df in tqdm(zip(image_paths, txt_paths if not _type == "test" else image_paths), total=len(image_paths)):
        source_image = cv2.imread(image_path, cv2.IMREAD_COLOR)
        image_height, image_width, _ = source_image.shape # 1024 1024 3

        target_image_path = f"data/yolo/{_type}/{os.path.basename(image_path)}"
        cv2.imwrite(target_image_path, source_image)

        if _type == "test":
            continue


        yolo_labels = []
        for line in labels_df:
                line = ast.literal_eval(line)

                line = [item for sublist in line for item in sublist]

                class_name = 0 #모두 싹다 정유탱크, class-name=1
                x_min, y_min = float(min(line[4], line[6])), float(min(line[5], line[7]))
                x_max, y_max = float(max(line[0], line[2])), float(max(line[1], line[3]))
                x, y = float(((x_min + x_max) / 2) / image_width), float(((y_min + y_max) / 2) / image_height)
                w, h = abs(x_max - x_min) / image_width, abs(y_max - y_min) / image_height
                yolo_labels.append(f"{class_name} {x} {y} {w} {h}")

        target_label_txt = f"data/yolo/{_type}/{image_path.split('/')[-1][:-3]+'txt'}" #이름 같게
        with open(target_label_txt, "w") as writer:
            for yolo_label in yolo_labels:
                writer.write(f"{yolo_label}\n")

In [10]:
image_paths = sorted(glob("/content/oiltank_dataset/train_images/*.png"))

txt_paths = train_df.groupby('train_data_loc').apply(lambda x: x['train_label_xy'].tolist())

train_images_paths, valid_images_paths, train_txt_paths, valid_txt_paths = train_test_split(image_paths, txt_paths, test_size=0.1, random_state=42)

make_yolo_dataset(train_images_paths, train_txt_paths, "train")
make_yolo_dataset(valid_images_paths, valid_txt_paths, "valid")
make_yolo_dataset(sorted(glob("/content/oiltank_dataset/valid_images/*.png")), None, "test")

100%|██████████| 68/68 [00:07<00:00,  9.19it/s]


#train

In [11]:
yaml_data = {
              "names": ['oiltank'],
              "nc": 1,
              "path": "/content/data/yolo/",
              "train": "train",
              "val": "valid",
              "test": "test"
            }

with open("data/yolo/custom.yaml", "w") as writer:
    yaml.dump(yaml_data, writer)

In [47]:

SEED = 42
BATCH_SIZE = 3 #컴터 환경 따라. 높을수록 좋음(아마?)
MODEL = "first_model"


#model = YOLO(f"{MODEL}/train/weights/last.pt")
model = YOLO("yolov8x")

results = model.train(
    data="/content/data/yolo/custom.yaml",
    imgsz=1024,
    epochs=200,
    batch=BATCH_SIZE,
    patience=5,
    workers=2, #컴터 환경 따라
    device=0, #컴터 환경 따라
    exist_ok=True,
    project=f"{MODEL}",
    name="train",
    seed=SEED,
    pretrained=False, #pretrained사용 불가, 직접 학습해야 함.
    resume=False, #계속 학습 여부
    optimizer="Adam",
    lr0=1e-3,
    augment=False, #true하면 오류남.
    val=True,
    cache=True
    )

Ultralytics YOLOv8.0.134 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
yolo/engine/trainer: task=detect, mode=train, model=yolov8x.pt, data=/content/data/yolo/custom.yaml, epochs=200, patience=5, batch=3, imgsz=1024, save=True, save_period=-1, cache=True, device=0, workers=2, project=first_model, name=train, exist_ok=True, pretrained=False, optimizer=Adam, verbose=True, seed=42, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=torchscript, keras=False, optimize=False, int8=Fals

#inference

In [14]:
def get_test_image_paths(test_image_paths):
    for i in range(0, len(test_image_paths), BATCH_SIZE):
        yield test_image_paths[i:i+BATCH_SIZE]

In [24]:
model = YOLO(f"/content/{MODEL}/train/weights/best.pt")
test_image_paths = glob("/content/data/yolo/test/*.png")
for i, image in tqdm(enumerate(get_test_image_paths(test_image_paths)), total=int(len(test_image_paths)/BATCH_SIZE)):
    model.predict(image, imgsz=(1024, 1024), iou=0.5, conf=0.5, save_conf=True, save=False, save_txt=True, project=f"{MODEL}", name="predict",
                  exist_ok=True, device=0, augment=True, verbose=False)
    if i % 5 == 0:
        clear_output(wait=True)

 94%|█████████▍| 16/17 [00:30<00:01,  1.83s/it]Results saved to first_model/predict
68 labels saved to first_model/predict/labels
100%|██████████| 17/17 [00:32<00:00,  1.89s/it]


In [25]:
def yolo_to_labelme(line, image_width, image_height, txt_file_name):
    file_name = txt_file_name.split("/")[-1].replace(".txt", ".png")
    class_id, x, y, width, height, confidence = [float(temp) for temp in line.split()]

    x_min = int((x - width / 2) * image_width)
    x_max = int((x + width / 2) * image_width)
    y_min = int((y - height / 2) * image_height)
    y_max = int((y + height / 2) * image_height)

    return file_name, int(class_id), confidence, x_min, y_min, x_max, y_min, x_max, y_max, x_min, y_max

In [41]:
infer_txts = glob(f"{MODEL}/predict/labels/*.txt")

results = []
for infer_txt in tqdm(infer_txts):

    base_file_name = infer_txt.split("/")[-1].split(".")[0]
    imgage_height, imgage_width = cv2.imread(f"data/yolo/test/{base_file_name}.png").shape[:2]
    with open(infer_txt, "r") as reader:
        lines = reader.readlines()
        for line in lines:
            results.append(yolo_to_labelme(line, imgage_width, imgage_height, infer_txt))

df_submission = pd.DataFrame(data=results, columns=["file_name", "class_id", "confidence", "point1_x", "point1_y", "point2_x", "point2_y", "point3_x", "point3_y", "point4_x", "point4_y"])

100%|██████████| 68/68 [00:01<00:00, 35.87it/s]


In [42]:
#File	Confidence	X1	Y1	X2	Y2	X3	Y3	X4	Y4

df_submission['confidence']=0.1
df_submission.drop(columns=['class_id'],inplace=True)
df_submission.rename(columns={
    'point1_x': 'X1',
    'point2_x': 'X2',
    'point3_x': 'X3',
    'point4_x': 'X4',
    'point1_y': 'Y1',
    'point2_y': 'Y2',
    'point3_y': 'Y3',
    'point4_y': 'Y4',
    'confidence' : 'Confidence',
    'file_name' : 'File'
    }, inplace=True)

df_submission['File'] = df_submission['File'].str[:8]

In [43]:
df_submission

,File,Confidence,X1,Y1,X2,Y2,X3,Y3,X4,Y4
0,OBJ03894,0.1,538,499,566,499,566,529,538,529
1,OBJ03894,0.1,482,506,511,506,511,536,482,536
2,OBJ03894,0.1,510,475,538,475,538,505,510,505
3,OBJ03894,0.1,14,6,52,6,52,43,14,43
4,OBJ03894,0.1,483,450,512,450,512,479,483,479
...,...,...,...,...,...,...,...,...,...,...
4271,OBJ05372,0.1,159,922,245,922,245,1002,159,1002
4272,OBJ05372,0.1,144,193,174,193,174,209,144,209
4273,OBJ05372,0.1,107,177,165,177,165,186,107,186
4274,OBJ05372,0.1,310,874,365,874,365,886,310,886


In [44]:
df_submission.to_csv(f"results.csv", index=False)